In [55]:
import pandas as pd
import gensim
import nltk
import gensim.corpora as corpora
from gensim.models import LdaModel, CoherenceModel, Phrases
from gensim.corpora import Dictionary
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk import pos_tag
from nltk.corpus import wordnet
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis
import re

In [56]:
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('wordnet')

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\meet9\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\meet9\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [57]:
import pandas as pd
data = pd.read_csv('dataset_expedia-hotels-com-reviews-scraper_2024.csv')

In [58]:
data.head()

,__typename,brandType,contentDirectFeedbackPromptId,customData/doWeEndorseIt,customData/hotel,hotelId,id,impressionAnalytics,impressionAnalytics/__typename,impressionAnalytics/event,...,translationInfo/loadingTranslationText,translationInfo/seeOriginalText,translationInfo/targetLocale,translationInfo/translatedBy/__typename,translationInfo/translatedBy/description,translationInfo/translationCallToActionLabel,travelers/0,travelers/1,travelers/2,travelers/3
0,PropertyReview,Expedia,NaN,no opinion,Prague Hotel Krystal,428588,66b63b958397934320ad7f75,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Traveled with family and small children,Traveled with family,NaN,NaN
1,PropertyReview,Expedia,NaN,no opinion,Prague Hotel Krystal,428588,66b503566a7cfe1353bdb91e,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Traveled with family,NaN,NaN,NaN
2,PropertyReview,Expedia,NaN,no opinion,Prague Hotel Krystal,428588,66b4d8d6e6073643fd1b9cdc,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Traveled with family and small children,NaN,NaN,NaN
3,PropertyReview,Expedia,NaN,no opinion,Prague Hotel Krystal,428588,66b3c0f33677b72a4489b05f,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Traveled with family and small children,NaN,NaN,NaN
4,PropertyReview,Expedia,NaN,no opinion,Prague Hotel Krystal,428588,66b36f292fa7b179c05b6637,NaN,ClientSideImpressionEventAnalytics,Scroll reviews,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
data.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2412 entries, 0 to 2411
Data columns (total 120 columns):
 #    Column                                                    Dtype  
---   ------                                                    -----  
 0    __typename                                                object 
 1    brandType                                                 object 
 2    contentDirectFeedbackPromptId                             float64
 3    customData/doWeEndorseIt                                  object 
 4    customData/hotel                                          object 
 5    hotelId                                                   int64  
 6    id                                                        object 
 7    impressionAnalytics                                       float64
 8    impressionAnalytics/__typename                            object 
 9    impressionAnalytics/event                                 object 
 10   impressionAnalytics/re

In [60]:
data.shape

(2412, 120)

In [90]:
# Step 1: Load default NLTK stopwords for English and Spanish
stop_words_en = set(stopwords.words('english'))  # Convert list to set
stop_words_es = set(stopwords.words('spanish'))  # Convert list to set

# List of negative words to remove from the stop words list
negative_words = [
    'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 
    'don', "don't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', 
    "isn't", 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'no', 'nor', 
    'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't",
    "won't", 'wouldn', "wouldn't"
]

# Step 2: Modify the English stopwords list
custom_stop_words_en = stop_words_en.copy()  # Create a copy to modify

# Remove negative words from the custom stop words list
for word in negative_words:
    custom_stop_words_en.discard(word)  # discard() avoids KeyError if the word is not found

# Optionally, add or remove other words as needed
custom_stop_words_en.add('example')  # Example modification

# Step 3: Combine the modified English stopwords with Spanish stopwords
stop_words = custom_stop_words_en.union(stop_words_es)  # Combine sets using .union()

lemmatizer_en = WordNetLemmatizer()
stemmer_es = SnowballStemmer('spanish')

# 1. Preprocessing Function (Simplified with Bigram within the function)
def preprocess(text):
    if not isinstance(text, str):
        text = ''
    
    # Remove special characters and numbers
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    
    # Convert to lowercase and tokenize
    tokens = simple_preprocess(text.lower(), deacc=True)
    
    # Lemmatize or stem and remove stopwords
    processed_tokens = []
    for word in tokens:
        if word not in stop_words:
            if word in custom_stop_words_en:
                processed_tokens.append(lemmatizer_en.lemmatize(word))
            elif word in stop_words_es:
                processed_tokens.append(stemmer_es.stem(word))
            else:
                processed_tokens.append(word)
    
    # Create bigrams
    bigram = Phrases([processed_tokens], min_count=5, threshold=100)
    tokens = bigram[processed_tokens]
    
    return tokens

In [91]:
#custom_stop_words_en = stop_words_en.copy()
#sorted_custom_stop_words_en = sorted(stop_words_en)
#print(sorted_custom_stop_words_en)

In [92]:
# Apply preprocessing
data['processed_text'] = data['text'].apply(preprocess)

In [93]:
print(data['text'][0])

The property was too large for my liking. The food was okay some days and not great at all some days. Definitely the food needs a upgrade especially the Italian food. The pool area was beautiful, but the pools had mold. Pools need to be cleaned better. and the Beach lets talk about the beach. Its horrible it has tons of sea weed. If you are going to this hotel for the beach forget it. reroute yourself else were. I would give this hotel a 3 out of 5. Maybe next time I would prefer the Grand. Its newer.


In [94]:
print(data['processed_text'][0])

['property', 'large', 'liking', 'food', 'okay', 'days', 'great', 'days', 'definitely', 'food', 'needs', 'upgrade', 'especially', 'italian', 'food', 'pool', 'area', 'beautiful', 'pools', 'mold', 'pools', 'need', 'cleaned', 'better', 'beach', 'lets', 'talk', 'beach', 'horrible', 'tons', 'weed', 'going', 'hotel', 'beach', 'forget', 'reroute', 'else', 'would', 'give', 'hotel', 'maybe', 'next', 'time', 'would', 'prefer', 'grand', 'newer']


In [95]:
# Create Dictionary and Corpus
dictionary = corpora.Dictionary(data['processed_text'])
# Filter out extremes
dictionary.filter_extremes(no_below=5, no_above=0.5)
corpus = [dictionary.doc2bow(text) for text in data['processed_text']]

In [96]:
# Build LDA Model
num_topics = 4

lda_model = LdaModel(corpus=corpus,
                     id2word=dictionary,
                     num_topics=num_topics, 
                     random_state=100,
                     update_every=1,
                     chunksize=4000,
                     passes=10,
                     alpha='auto',
                     eta='auto',
                     per_word_topics=True)

In [98]:
# 4. Print the 25 most significant terms per topic
for i in range(num_topics):
    print(f'Topic {i+1}:')
    print([term for term, _ in lda_model.show_topic(i, topn=25)])
    print('\n')

Topic 1:
['hotel', 'great', 'property', 'excelente', 'servicio', 'everything', 'place', 'food', 'staff', 'best', 'clean', 'friendly', 'personal', 'atencion', 'comida', 'service', 'bien', 'super', 'restaurantes', 'beautiful', 'resort', 'gusto', 'amazing', 'playa', 'need']


Topic 2:
['room', 'resort', 'us', 'get', 'time', 'hotel', 'check', 'service', 'would', 'staff', 'day', 'one', 'food', 'back', 'great', 'like', 'go', 'also', 'good', 'didn', 'experience', 'never', 'property', 'nice', 'even']


Topic 3:
['staff', 'food', 'time', 'us', 'pool', 'good', 'like', 'place', 'back', 'beach', 'great', 'go', 'property', 'would', 'stay', 'service', 'get', 'amazing', 'nice', 'definitely', 'didn', 'room', 'one', 'helpful', 'friendly']


Topic 4:
['food', 'resort', 'staff', 'good', 'beach', 'great', 'palace', 'service', 'amazing', 'moon', 'sunrise', 'excellent', 'options', 'water', 'property', 'restaurants', 'pool', 'stay', 'friendly', 'clean', 'time', 'nice', 'grand', 'kids', 'pools']




In [99]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

# Visualize the topics using pyLDAvis
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, dictionary, sort_topics=False)

# Display the pyLDAvis visualization
pyLDAvis.display(vis)

In [73]:
# Get the topic distribution for each document
topic_distributions = []
for bow in corpus:
    topic_distribution = lda_model.get_document_topics(bow, minimum_probability=0)
    topic_distributions.append([weight for _, weight in topic_distribution])

# Create a DataFrame
topic_df = pd.DataFrame(topic_distributions, columns=[f'Topic_{i+1}' for i in range(lda_model.num_topics)])

# Add the processed text to the DataFrame
topic_df.insert(0, 'Processed_Text', data['processed_text'])

# Add a "Dominant_Topic" column with the name of the topic with the highest weight
topic_df['Dominant_Topic'] = topic_df.iloc[:, 1:].idxmax(axis=1)

topic_df

,Processed_Text,Topic_1,Topic_2,Topic_3,Dominant_Topic
0,"[property, large, liking, food, okay, days, gr...",0.879399,0.117467,0.003134,Topic_1
1,"[stuff, friendly, food, excellent, variety, fo...",0.985585,0.008249,0.006166,Topic_1
2,"[big, organized, transportation, short, staffe...",0.959393,0.023249,0.017358,Topic_1
3,[good],0.819412,0.103251,0.077337,Topic_1
4,"[really, pushy, comes, selling, resorts, examp...",0.978552,0.012342,0.009106,Topic_1
...,...,...,...,...,...
2407,"[complejo, lujo, atencion, personal, cada, lug...",0.038866,0.948679,0.012455,Topic_2
2408,"[tres, bien, entretenu, personnel, tres, ecout...",0.519789,0.431207,0.049005,Topic_1
2409,"[hicieron, validos, usd, habian, dado, pregunt...",0.090367,0.038184,0.871449,Topic_3
2410,[],0.572075,0.244406,0.183520,Topic_1


In [74]:
topic_df['Dominant_Topic'].value_counts()

Dominant_Topic
Topic_1    1875
Topic_2     356
Topic_3     181
Name: count, dtype: int64